## MODEL TRAINING
 Model objective: Predict math score based on dataset variables.
 

Import libraries for model training

In [2]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


In [4]:
df = pd.read_csv('data/stud.csv')

In [5]:
# create predictors data set
X = df.drop(columns=['math_score'],axis=1 ) # axis = 1 for column(col wise operation) , axis=0 for index (row wise ioperation)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
# create target variable 
Y=df['math_score']

In [7]:
Y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [18]:
X.dtypes

gender                         object
race_ethnicity                 object
parental_level_of_education    object
lunch                          object
test_preparation_course        object
reading_score                   int64
writing_score                   int64
dtype: object

In [25]:
# Separate pipelines for  numerical and categorical columns

num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns



In [56]:
X.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   reading_score                1000 non-null   int64 
 6   writing_score                1000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [57]:
X.shape 

(1000, 7)

In [27]:
cat_features

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')

In [59]:
#train test spilt 80/20
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape,Y_train.shape,Y_test.shape

((800, 7), (200, 7), (800,), (200,))

<bound method NDFrame.head of 29     62
535    66
695    79
557    61
836    73
       ..
106    87
270    69
860    53
435    50
102    85
Name: math_score, Length: 800, dtype: int64>

In [31]:
#preprocess 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
ohe_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", ohe_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [71]:
# VARIABLE TRANSFORM MODEL FOR TRAIN AND TEST VARIABLES
Preprocess_model=preprocessor.fit(X_train)



In [78]:
#transform X_train and X_test on 
X_train_transformed = Preprocess_model.transform(X_train)
X_test_transformed = Preprocess_model.transform(X_test)

In [82]:
##  MODEL EVALUATION METRICS

#mae,rmse,rsquare

def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    rmse= np.sqrt(mean_squared_error(true,predicted))
    r2 = r2_score(true,predicted)
    return mae,rmse,r2

In [86]:
#models to use

models = { 
 "Linear Regression":LinearRegression(),
 "Lasso": Lasso(),
 "Ridge": Ridge(),
 "KNN" : KNeighborsRegressor(),
 "Decision Tree": DecisionTreeRegressor(),
 "Random Forest": RandomForestRegressor(),
 "XGB":XGBRegressor(),
 "Catboost":CatBoostRegressor(verbose=False),
 "AdaBoost":AdaBoostRegressor()
}

model_list=[]
r2_list=[]
mae_list=[]
rmse_list=[]


for i in range(len(models)):
    #train
    model=list(models.values())[i]
    model.fit(X_train_transformed,Y_train)
    
    #Predict
    y_train_pred = model.predict(X_train_transformed)
    y_test_pred  = model.predict(X_test_transformed) 
    
    # Evaluate
    train_mae,train_rmse,train_r2 = evaluate_model(Y_train,y_train_pred)
    test_mae,test_rmse,test_r2 = evaluate_model(Y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('MODEL PERFORMANCE FOR TRAINING SET')
    print("- MAE : {:.4f} ".format(train_mae))
    print("- RMSE : {:.4f} ".format(train_rmse))
    print("- R2: {:.4f} ".format(train_r2))
    print('-'*35) 
    print('MODEL PERFORMANCE FOR TEST SET')
    print("- MAE : {:.4f} ".format(test_mae))
    print("- RMSE : {:.4f} ".format(test_rmse))
    print("- R2: {:.4f} ".format(test_r2))
    
    r2_list.append(test_r2)
    mae_list.append(test_mae)
    rmse_list.append(test_rmse)

    print('='*35)
    print('\n')

Linear Regression
MODEL PERFORMANCE FOR TRAINING SET
- MAE : 4.2687 
- RMSE : 5.3250 
- R2: 0.8742 
-----------------------------------
MODEL PERFORMANCE FOR TEST SET
- MAE : 4.2335 
- RMSE : 5.4201 
- R2: 0.8793 


Lasso
MODEL PERFORMANCE FOR TRAINING SET
- MAE : 5.2053 
- RMSE : 6.5925 
- R2: 0.8072 
-----------------------------------
MODEL PERFORMANCE FOR TEST SET
- MAE : 5.1557 
- RMSE : 6.5173 
- R2: 0.8254 


Ridge
MODEL PERFORMANCE FOR TRAINING SET
- MAE : 4.2650 
- RMSE : 5.3233 
- R2: 0.8743 
-----------------------------------
MODEL PERFORMANCE FOR TEST SET
- MAE : 4.2111 
- RMSE : 5.3904 
- R2: 0.8806 


KNN
MODEL PERFORMANCE FOR TRAINING SET
- MAE : 4.5025 
- RMSE : 5.6941 
- R2: 0.8562 
-----------------------------------
MODEL PERFORMANCE FOR TEST SET
- MAE : 5.5840 
- RMSE : 7.2148 
- R2: 0.7861 


Decision Tree
MODEL PERFORMANCE FOR TRAINING SET
- MAE : 0.0187 
- RMSE : 0.2795 
- R2: 0.9997 
-----------------------------------
MODEL PERFORMANCE FOR TEST SET
- MAE : 6.4